<a href="https://colab.research.google.com/github/LeslieLeeGithub/42173_Assignment2_Literature_Review/blob/main/T5_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Somehow I can not use load_metric eventhough I tryied pip install datasets for several times. Thus I changed to use evaluate package to compute rougescore. In that case, can not display rouge2 recall correctly, use rouge 2 score instead .*

In [80]:
# from datasets import load_metric

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)

# Section 1 : Installing and importing required libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pwd

'/content'

In [5]:
cd /content/drive/MyDrive/42173_Advanced_Natural_Language_Processing

/content/drive/MyDrive/42173_Advanced_Natural_Language_Processing


In [7]:
!pip install transformers datasets evaluate


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import torch

2. Load the Pre-trained T5 Model and Tokenizer
The T5 model and tokenizer are available via Hugging Face's transformers library. We’ll use t5-small here for simplicity, but you can also choose larger versions like t5-base or t5-large for better performance.

In [7]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')


3. Load the CNN/DailyMail Dataset
The CNN/DailyMail dataset is one of the most popular datasets for text summarization, containing news articles paired with human-written summaries.

You can easily load it using the Hugging Face datasets library:

In [8]:
# from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Take a look at the dataset structure
print(dataset['train'][0])


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

The dataset will contain columns like:

article: The full news article.
highlights: The human-written summary (ground truth).

4. Preprocess the Dataset
We need to prepare the dataset by tokenizing the input text (the news article) and the target text (the summary). T5 requires that we prepend the task type (summarize:) to the input.



In [13]:
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True, return_tensors="pt")

    # Tokenize the targets (summaries)
    labels = tokenizer(examples['highlights'], max_length=150,  padding=True, truncation=True, return_tensors="pt").input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

This step converts the input articles and summaries into the format required by the T5 model. You can adjust the max_length to fit your needs and computational resources.

5. Fine-tuning the T5 Model
We’ll use Hugging Face’s Trainer API, which simplifies the training process. First, define the training arguments:



In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-summarization",        # Where to save the model
    evaluation_strategy="epoch",            # Evaluate after every epoch
    per_device_train_batch_size=4,          # Adjust based on your GPU memory
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,                     # Only save the best 2 models
    num_train_epochs=3,                     # Number of training epochs
    save_strategy="epoch",                  # Save model after each epoch
    load_best_model_at_end=True,            # Load the best model after training
)

# Initialize the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


This configuration sets up training parameters such as the number of epochs, batch size, and evaluation strategy. You can adjust per_device_train_batch_size based on your available GPU memory.

6. Start Training
Now, you can start training the model:



In [17]:
# Start fine-tuning
trainer.train()


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

The model will be fine-tuned on the CNN/DailyMail dataset for abstractive summarization. After each epoch, it will evaluate the performance on the validation set.

7. Evaluate the Model
Once training is complete, you can evaluate the model using summarization-specific metrics like ROUGE. Hugging Face provides an easy way to compute this with the evaluate library:



In [ ]:
import evaluate

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Generate predictions on the test set
predictions = trainer.predict(tokenized_datasets['test'])

# Decode the generated summaries
preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Calculate ROUGE scores
results = rouge.compute(predictions=preds, references=tokenized_datasets['test']['highlights'])
print(results)


The ROUGE metric compares the generated summaries to the ground truth summaries by measuring n-gram overlaps, which is a standard evaluation method in summarization tasks.



8. Generate Summaries on New Data
Once the model is fine-tuned, you can generate summaries for new documents by passing input texts through the model:

In [ ]:
def summarize(text):
    inputs = tokenizer(f"summarize: {text}", return_tensors="pt", truncation=True, max_length=512)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example
article = "The stock market experienced a significant downturn today with major indices falling by more than 2%..."
print(summarize(article))


9. Save the Fine-Tuned Model
After training and fine-tuning, save the model to a directory so you can use it later:

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./t5-summarization-model')
tokenizer.save_pretrained('./t5-summarization-model')


---

# Section 2 : Loading and preprocessing the dataset

## trying to use XSum dataset(Extreme Summarization Dataset)

Description: XSum is a dataset for abstractive summarization where each article is paired with a highly concise one-sentence summary. This dataset contains BBC articles and is widely used to test models' ability to generate short, concise summaries.
\
\
Task: Summarizing a news article into a one-sentence summary.
\
\
Dataset Size: 204,045 training, 11,332 validation, 11,334 test examples.
\
\
Recommended for: Models that need to generate very short summaries that capture the main point.


In [35]:
# Load XSum dataset using Hugging Face's datasets library
dataset = load_dataset("xsum")


In [12]:
# Check the size of the training set
print(len(dataset['train']))

204045


In [15]:
# Print an example from the training set
print(dataset['train'][0])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [36]:
# # Selecting 5000 records for training, 1000 for testing, and using full validation set
# train_dataset = dataset['train'].select(range(5000))
# test_dataset = dataset['test'].select(range(1000))
# validation_dataset = dataset['validation']  # Keep the full validation set

# Selecting 5000 records for training, 1000 for testing, and using full validation set
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']  # Keep the full validation set

In [37]:

# Load the pre-trained T5 model and tokenizer (using 't5-small', but you can use 't5-base', 't5-large', etc.)
MODEL_ID = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(MODEL_ID)
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID)

In [38]:
# Enable gradient checkpointing to reduce memory consumption
model.gradient_checkpointing_enable()

In [39]:
# Preprocess the dataset: Add "summarize:" to the input and tokenize
def preprocess_function(examples):
    # Add "summarize:" prefix to each article
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize the summaries (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    # Add tokenized summaries as labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess the training, validation, and test datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True)

# Now, you have tokenized datasets ready for fine-tuning:
# tokenized_train, tokenized_test, and tokenized_validation

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

# Section 3 : Evaluating the pre trained model before fine tuning

In [20]:
# !pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fcc0c7336fb1a99977120179f6f14ed6f84a6486d463e4f963cedc5d6b023404
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [18]:
import evaluate

In [21]:
# Load the ROUGE metric from evaluate
rouge = evaluate.load("rouge")

In [41]:

# Function to generate summaries from the test data and compute ROUGE score
def generate_summary(batch, model, tokenizer):
    # Prepare inputs for the model with the "summarize:" prefix
    inputs = tokenizer("summarize: " + batch["document"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(DEVICE)  # Move input_ids to the device (GPU/CPU)
    attention_mask = inputs.attention_mask.to(DEVICE)  # Move attention_mask to the device

    # Generate summaries using T5 model
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode generated summaries into human-readable text
    batch["predicted_summary"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Clear CUDA cache after each batch to free up memory
    torch.cuda.empty_cache()
    return batch

# Set the device to GPU if available, otherwise use CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)

# Load the pre-trained T5 model
MODEL_ID = "t5-small"  # You can use t5-base or t5-large for better performance
pretrained_model = T5ForConditionalGeneration.from_pretrained(MODEL_ID).to(DEVICE)

# Select a small subset of the test data for evaluation (e.g., 100 examples)
test_data_sample_pretrain = tokenized_test.select(range(100)).map(lambda batch: generate_summary(batch, pretrained_model, tokenizer), batched=False)




ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['prison link Cymru says some ex-offenders were living rough for up to a year before finding suitable accommodation. change to the Housing Act in Wales introduced in 2015 removed the right for prison leavers to be given priority for accommodation. the charity says the need for accommodation is "chronic"'],
Input references: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.

In [46]:

# Ensure predictions and references are lists of strings, not lists of lists
predictions = [" ".join(pred) if isinstance(pred, list) else pred for pred in test_data_sample_pretrain["predicted_summary"]]
references = [" ".join(ref) if isinstance(ref, list) else ref for ref in test_data_sample_pretrain["summary"]]

# Debug: Check if predictions and references have the same length
print(f"Number of predictions: {len(predictions)}")
print(f"Number of references: {len(references)}")

# Debug: Check the format of a few predictions and references
print(f"Sample prediction: {predictions[0]}")
print(f"Sample reference: {references[0]}")



# Compute ROUGE scores for the pre-trained model
rouge_scores_pretrain = rouge.compute(predictions=predictions, references=references)

# Print the scalar ROUGE scores (since they are likely scalar values)
for key, value in rouge_scores_pretrain.items():
    print(f"{key}:")
    print(f"  Score: {value:.4f}")


Number of predictions: 100
Number of references: 100
Sample prediction: prison link Cymru says some ex-offenders were living rough for up to a year before finding suitable accommodation. change to the Housing Act in Wales introduced in 2015 removed the right for prison leavers to be given priority for accommodation. the charity says the need for accommodation is "chronic"
Sample reference: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
rouge1:
  Score: 0.1808
rouge2:
  Score: 0.0279
rougeL:
  Score: 0.1303
rougeLsum:
  Score: 0.1298


note:
\
The ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metrics are commonly used to evaluate the quality of machine-generated text, such as summaries, by comparing them to human-written reference summaries. It measures how many words or phrases overlap between the machine-generated text and the reference text. Here's a breakdown of the different ROUGE metrics:

1. ROUGE-1
Definition: ROUGE-1 refers to the overlap of unigrams (single words) between the generated summary and the reference summary.
Interpretation: It measures how many single words in the generated summary are also in the reference summary. A higher ROUGE-1 score indicates that more words in the generated summary match the reference summary.
2. ROUGE-2
Definition: ROUGE-2 measures the overlap of bigrams (two consecutive words) between the generated and reference summaries.
Interpretation: It assesses how well the generated summary captures word sequences (pairs of words) that appear in the reference summary. ROUGE-2 is more difficult to score well on compared to ROUGE-1 because it requires that both words and their order match in pairs.
3. ROUGE-L
Definition: ROUGE-L is based on the Longest Common Subsequence (LCS). It measures the longest sequence of words that appear in both the generated and reference summaries, in the same order.
Interpretation: ROUGE-L takes into account the sentence structure and reflects how well the overall sequence of words in the generated summary aligns with the reference. It rewards longer matching sequences of words, so it’s a more structural measure than ROUGE-1 or ROUGE-2.
4. ROUGE-LSum
Definition: ROUGE-LSum is a variant of ROUGE-L that’s designed for summarization tasks. It considers the Longest Common Subsequence across multiple sentences, making it more appropriate for multi-sentence summarization tasks.
Interpretation: ROUGE-LSum looks at the longest subsequence of words across the entire summary (multiple sentences) rather than just within individual sentences.
What These Scores Mean in Your Output:
ROUGE-1 score (0.1808): This means that 18.08% of the unigrams in your generated summaries overlap with the reference summaries. In other words, about 18% of the single words match.
ROUGE-2 score (0.0279): Only 2.79% of the bigrams (two consecutive words) in your generated summaries match the reference summaries. This lower score suggests that your model is not doing as well at capturing the correct word sequences.
ROUGE-L score (0.1303): This score indicates that about 13.03% of the longest common subsequences match between your generated and reference summaries. It reflects how well the overall structure of the generated summaries matches the reference.
ROUGE-LSum score (0.1298): This is very close to the ROUGE-L score and reflects a similar level of matching when considering the overall sequence of words across multiple sentences in the summaries.
Summary of ROUGE Metrics:
ROUGE-1: Measures word-level overlap.
ROUGE-2: Measures bigram (two-word sequence) overlap.
ROUGE-L: Measures the longest matching sequence of words, capturing sentence structure.
ROUGE-LSum: Specialized for summarization, measuring the longest common subsequence across multiple sentences.
Improving Scores:
Low ROUGE-2 score** suggests that the model might struggle with generating coherent word sequences, which could be improved by better fine-tuning on the task.
ROUGE-1 gives a general idea of the word overlap, but higher-order n-grams like ROUGE-2 give insight into how well the generated summary matches the reference at the level of fluency and coherence.

# Section 4: Defining the arguments and fine tuning the model , then evaluating the fine tuned model

i ran the code for training by using the whole Xsum dataset, and it said that will use more than 4hours to finish the fine tuning process.

however, when i was trying to do different version in this section, i delete it by mistake. \
any way, that part of code is not important

In [57]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Set training arguments with evaluation strategy
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate after each epoch using validation set
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer with train, validation, and test sets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,  # Use validation set for evaluation
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

# Optionally evaluate on the test set after fine-tuning
print("Evaluating on the test set...")
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [id, summary, document]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

# Section 5, since the whole training will cost 4 hours, then split it into small sample

In [72]:
# Apply the preprocessing function to the training, validation, and test datasets
train_dataset = dataset['train'].select(range(5000)).map(preprocess_function, batched=True)
validation_dataset = dataset['validation'].select(range(1000)).map(preprocess_function, batched=True)
test_dataset = dataset['test'].select(range(1000)).map(preprocess_function, batched=True)

In [73]:

# Preprocess the training, validation, and test datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

calculate the rougue score before fine tuning

In [74]:
# Set the device to GPU if available, otherwise use CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)

# Load the pre-trained T5 model
MODEL_ID = "t5-small"  # You can use t5-base or t5-large for better performance
pretrained_model = T5ForConditionalGeneration.from_pretrained(MODEL_ID).to(DEVICE)

# Select a small subset of the test data for evaluation
test_data_sample_pretrain = tokenized_test.select(range(100)).map(lambda batch: generate_summary(batch, pretrained_model, tokenizer), batched=False)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [75]:
# Ensure predictions and references are lists of strings, not lists of lists
predictions = [" ".join(pred) if isinstance(pred, list) else pred for pred in test_data_sample_pretrain["predicted_summary"]]
references = [" ".join(ref) if isinstance(ref, list) else ref for ref in test_data_sample_pretrain["summary"]]

# Debug: Check if predictions and references have the same length
print(f"Number of predictions: {len(predictions)}")
print(f"Number of references: {len(references)}")

# Debug: Check the format of a few predictions and references
print(f"Sample prediction: {predictions[0]}")
print(f"Sample reference: {references[0]}")



# Compute ROUGE scores for the pre-trained model
rouge_scores_pretrain = rouge.compute(predictions=predictions, references=references)

# Print the scalar ROUGE scores (since they are likely scalar values)
for key, value in rouge_scores_pretrain.items():
    print(f"{key}:")
    print(f"  Score: {value:.4f}")


Number of predictions: 100
Number of references: 100
Sample prediction: prison link Cymru says some ex-offenders were living rough for up to a year before finding suitable accommodation. change to the Housing Act in Wales introduced in 2015 removed the right for prison leavers to be given priority for accommodation. the charity says the need for accommodation is "chronic"
Sample reference: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
rouge1:
  Score: 0.1808
rouge2:
  Score: 0.0279
rougeL:
  Score: 0.1303
rougeLsum:
  Score: 0.1298


fine tuning and evluating

In [78]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Define the preprocess function for summarization task
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize the summaries (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    # Add tokenized summaries as labels
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Preprocess the dataset
tokenized_train = dataset['train'].select(range(5000)).map(preprocess_function, batched=True)
tokenized_validation = dataset['validation'].select(range(1000)).map(preprocess_function, batched=True)
tokenized_test = dataset['test'].select(range(1000)).map(preprocess_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Use 'epoch' evaluation strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    remove_unused_columns=True  # Ensure unnecessary columns are removed
)

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
)

# Fine-tune the model on the smaller dataset
trainer.train()

# Evaluate on the test set after fine-tuning
print("Evaluating on the test set...")
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.778500,0.659522
2,0.735000,0.639191
3,0.714700,0.636264


Evaluating on the test set...


{'eval_loss': 0.6517480611801147, 'eval_runtime': 7.4895, 'eval_samples_per_second': 133.521, 'eval_steps_per_second': 16.69, 'epoch': 3.0}


In [79]:
# Generate predictions and compute ROUGE scores for the fine-tuned model
test_data_sample_finetuned = tokenized_test.select(range(100)).map(lambda batch: generate_summary(batch, model, tokenizer), batched=False)

predictions = [" ".join(pred) if isinstance(pred, list) else pred for pred in test_data_sample_finetuned["predicted_summary"]]
references = [" ".join(ref) if isinstance(ref, list) else ref for ref in test_data_sample_finetuned["summary"]]

# Compute ROUGE scores for the fine-tuned model
rouge_scores_finetuned = rouge.compute(predictions=predictions, references=references)
print("ROUGE Scores after Fine-Tuning:")
for key, value in rouge_scores_finetuned.items():
    print(f"{key}: {value:.4f}")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

ROUGE Scores after Fine-Tuning:
rouge1: 0.2515
rouge2: 0.0671
rougeL: 0.1932
rougeLsum: 0.1929
